In [3]:
import nltk
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import string
import requests
from bs4 import BeautifulSoup
import signal
from contextlib import contextmanager
import signal
from contextlib import contextmanager
from operator import itemgetter

class TimeoutException(Exception): pass
@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)
        
SIA = SentimentIntensityAnalyzer()

updated_words = {
    'buy':4,
    'gainer':4,
    'winner':4,
    'gain':3,
    'overweight':4,
    'cheap':3,
    'cheaper':3,
    'stable':2,
    'rise':3,
    'rises':3,    
    'rising':3,
    'growing':3,
    'grow':3,
    'growth':3,
    'bullish':4,    
    'bull':4,
    'undervalued':3,
    'long':3,
    'up':3,
    'climb':2,
    'climbing':2,    
    'surge':1,
    'surging':1,
    'surged':1,
    'popped':1,
    'dividends':3,
    'discount':3,   
    'cashed':2,
    'equity':2,
    'hold':3,
    'outperformed':3,
    'outperforms':3,    
    'outperform':3,
    'purchase':4,
    'strong':2,
    'stronger':2,
    'strongest':2,
    'sell':-4,   
    'expensive':-3,
    'risk':-2,
    'sank':-3,
    'sink':-3,
    'sunk':-3,
    'dip':-3,    
    'dipped':-3,
    'drop':-3,
    'bearish':-4,
    'bear':-4,
    'overvalued':-3,
    'overvalue':-3,   
    'underperformed':-3,
    'underperforms':-3,
    'underperform':-3,
    'short':-4,
    'unsustainable':-2,
    'unstable':-2,    
    'uncertain':-1,
    'uncertainty':-1,
    'down':-1,
    'lose':-3,
    'loser':-4,
    'inflation':-1,
    'reduce':-1,
    'reduced':-1,   
    'fell':-1,
    'fallen':-1,
    'falling':-1,
    'slowdown':-1,
    'recession':-1,    
    'plunges':-1,
    'plunge':-1,
    'decline':-1,
    'declined':-1,
    'declines':-1,
    'slow':-1,
    'cut':-1,
    'weak':-2,
    'weaker':-2,
    'weakest':-2,
    'slashed':-3,
    'slumped':-3,
    'pressure':-1,
    'miss':-1,
    'underweight':-4,
    'share':0,
    'shares':0,
    'amazon':0,
}

SIA.lexicon.update(updated_words)

stock = input("Input stock: ")
input_breakdown = []
{input_breakdown.append(x.lower()) for x in stock.split(" ")}
{SIA.lexicon.update({x:0}) for x in input_breakdown}

try:
    with time_limit(10):
        URL = f"https://news.google.com/search?q={stock}+stock+when=1d"
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
except TimeoutException as e:
    print("Timed out!")
        

text_list = soup.find_all('h3')
string_list = [ str(text).split('Aen">', 1)[1].split('</a>', 1)[0].replace(","," ") for text in text_list]


neutral_list = []
positive_list = []
negative_list = []

word_list = []
pos_word_list = []
neg_word_list = []

for text in string_list:
    score = SIA.polarity_scores(text)
    negative_list.append(score['neg'])
    neutral_list.append(score['neu'])
    positive_list.append(score['pos'])
    for word in text.split(" "):
        word_list.append(word)
        word_score = SIA.polarity_scores(word)
        if word_score['neg'] > word_score['pos']:
            neg_word_list.append(word.lower())
        if word_score['pos'] > word_score['neg']:
            pos_word_list.append(word.lower())

overall_neg = format(sum(negative_list)/len(negative_list),".3f")
overall_pos = format(sum(positive_list)/len(positive_list),".3f")
overall_neu = format(sum(neutral_list)/len(neutral_list),".3f")


word_list_count = []
{word_list_count.append([i,word_list.count(i)]) for i in word_list if [i,word_list.count(i)] not in word_list_count}
word_list_count = (sorted(word_list_count, key=itemgetter(1), reverse=True))

p_word_list_count = []
{p_word_list_count.append([i,pos_word_list.count(i)]) for i in pos_word_list if [i,pos_word_list.count(i)] not in p_word_list_count}
p_word_list_count = (sorted(p_word_list_count, key=itemgetter(1), reverse=True))

n_word_list_count = []
{n_word_list_count.append([i,neg_word_list.count(i)]) for i in neg_word_list if [i,neg_word_list.count(i)] not in n_word_list_count}
n_word_list_count = (sorted(n_word_list_count, key=itemgetter(1), reverse=True))

if (float(overall_pos)/float(overall_neg))>=1.75:
    print("The news are looking great!\n")
    print("These are some relevant words:")
    for x in range(0,5):
        try:
            print(f"{x+1}. {p_word_list_count[x][0]}")
        except:
            pass
if 1.75>(float(overall_pos)/float(overall_neg))>=1.25:
    print("The news are looking good!\n")
    print("These are some relevant words:")  
    for x in range(0,5):
        try:
            print(f"{x+1}. {p_word_list_count[x][0]}")
        except:
            pass
if 1.25>(float(overall_pos)/float(overall_neg))>0.75:
    print("The news are looking mixed\n")
if 0.75>(float(overall_pos)/float(overall_neg))>0.25:
    print("The news aren't looking good\n")
    print("These are some relevant words:")   
    for x in range(0,5):
        try:
            print(f"{x+1}. {n_word_list_count[x][0]}")
        except:
            pass
if 0.25>(float(overall_pos)/float(overall_neg)):
    print("The news are looking bad\n")
    print("These are some relevant words:")   
    for x in range(0,5):
        try:
            print(f"{x+1}. {n_word_list_count[x][0]}")
        except:
            pass
        
print("\nThese are some of the current news:")
for x in range(0,5):
    try:
        print(f"{x+1}. {string_list[x]}")
    except:
        pass

Input stock: Netflix
The news are looking good!

These are some relevant words:
1. growth
2. buy
3. worth
4. undervalued
5. boost

These are some of the current news:
1. Netflix stock jumps 6% after company revealed it only lost 970K subscribers last quarter
2. Netflix Stock: Ad-Supported Tier Could Boost Shares
3. Stock Market Today (7/20/22): Nasdaq Zips Higher as Netflix Earnings Impress
4. Netflix is going to start charging you $3 to share your password
5. Trade Netflix: Everything you need to know to buy or sell
